In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from TTS.tts.datasets import TTSDataset, load_tts_samples
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.tts.utils.visual import plot_pitch
from TTS.utils.audio import AudioProcessor

In [ ]:
f0_cache_path = "../../tests/data/ljspeech/f0_cache"

In [ ]:
from TTS.config.shared_configs import BaseAudioConfig, BaseDatasetConfig
from TTS.tts.configs.shared_configs import BaseTTSConfig

dataset = BaseDatasetConfig(formatter="ljspeech", meta_file_train="metadata.csv", path="../../tests/data/ljspeech")
CONFIG = BaseTTSConfig(
    datasets=[dataset],
    text_cleaner="basic_cleaners",
    audio=BaseAudioConfig(
        sample_rate=22050,
        do_trim_silence=True,
        trim_db=60.0,
        signal_norm=False,
        mel_fmin=0.0,
        mel_fmax=8000,
        spec_gain=1.0,
        log_func="np.log",
        ref_level_db=20,
        preemphasis=0.0,
    ),
)

# Alternatively, you can load an existing config file:
# from TTS.config import load_config
# CONFIG = load_config("path/to/config.json")

In [ ]:
ap = AudioProcessor.init_from_config(CONFIG)
tokenizer, CONFIG = TTSTokenizer.init_from_config(CONFIG)

train_samples, eval_samples = load_tts_samples(CONFIG.datasets, eval_split=False)
samples = train_samples + eval_samples

dataset = TTSDataset(
    samples=samples,
    ap=ap,
    tokenizer=tokenizer,
    compute_f0=True,
    f0_cache_path=f0_cache_path,
)

In [ ]:
idx = 1
item = dataset[idx]
wav = item["wav"]
pitch = item["pitch"]
spec = ap.melspectrogram(wav)
print("Text:", item["raw_text"])

In [ ]:
plot_pitch(pitch, spec.T)